# Домашнее задание к лекции "Базовые понятия статистики"

## Обязательная часть

Будем осуществлять работу с непростым [набором данных](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv) о состоянии здоровья лошадей, испытывающих кишечные колики. 

### Задание 1. Базовое изучение

Изучить представленный набор данных на основе [описания его столбцов](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.names) и выбрать 8 столбцов для дальнейшего изучения (среди них должны быть как числовые, так и категориальные). Провести расчет базовых метрик для них, кратко описать результаты.

### Задание 2. Работа с выбросами

В выбранных числовых столбцах найти выбросы, выдвинуть гипотезы об их причинах и проинтерпретировать результаты. Принять и обосновать решение о дальнейшей работе с ними.

### Задание 3. Работа с пропусками

Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе работы с пропусками по каждому столбцу, сформировать датафрейм, в котором пропуски будут отсутствовать.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('horse_data.csv', names=['surgery', 'age', 'hospital_number', 'rectal_temperature', 'pulse',
                                          'respiratory_rate', 'temperature_of_extremities', 'peripheral_pulse', 
                                          'mucous_membranes', 'capillary_refill_time', 'pain', 'peristalsis', 
                                          'abdominal_distension', 'nasogastric_tube', 'nasogastric_reflux',
                                          'nasogastric_reflux_ph', 'rectal_examination_feces', 'abdomen',
                                          'packed_cell_volume', 'total_protein', 'abdominocentesis_appearance',
                                          'abdomcentesis_total_protein', 'outcome', 'surgical_lesion', 'type_of_lesion_1',
                                          'type_of_lesion_2', 'type_of_lesion_3', 'cp_data'], na_values='?')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 28 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   surgery                      299 non-null    float64
 1   age                          300 non-null    int64  
 2   hospital_number              300 non-null    int64  
 3   rectal_temperature           240 non-null    float64
 4   pulse                        276 non-null    float64
 5   respiratory_rate             242 non-null    float64
 6   temperature_of_extremities   244 non-null    float64
 7   peripheral_pulse             231 non-null    float64
 8   mucous_membranes             253 non-null    float64
 9   capillary_refill_time        268 non-null    float64
 10  pain                         245 non-null    float64
 11  peristalsis                  256 non-null    float64
 12  abdominal_distension         244 non-null    float64
 13  nasogastric_tube    

In [4]:
df['outcome'].value_counts()

1.0    178
2.0     77
3.0     44
Name: outcome, dtype: int64

In [5]:
df['outcome'] = df['outcome'].apply(str).replace('1.0', 'lived').replace('2.0', 'died').replace('3.0', 'euthanized')

Более половины лошадей успешно прошли лечение. Около 15% были усыплены

In [6]:
df['surgery'] = df['surgery'].apply(str).str.replace('1.0', 'yes').replace('2.0', 'no')

In [7]:
df['surgery'].value_counts()

yes    180
no     119
nan      1
Name: surgery, dtype: int64

В 60% случаев на лошадях проводилась операция

In [8]:
df['age'] = df['age'].apply(str).replace('1', 'adult').replace('9', 'young')

In [9]:
df['age'].value_counts()

adult    276
young     24
Name: age, dtype: int64

Подавляющее большинство лошадей были взрослыми

In [10]:
df['hospital_number'] = df['hospital_number'].apply(str)

In [11]:
df['hospital_number'].isna().sum()

0

In [12]:
print(df['hospital_number'].value_counts().count())
print(df['hospital_number'].value_counts()[df['hospital_number'].value_counts() == 1].count())
print(df[df['hospital_number'].isin(
    df['hospital_number'].value_counts()[df['hospital_number'].value_counts() > 1].index)]['outcome'].value_counts())

284
268
lived         16
died          13
euthanized     3
Name: outcome, dtype: int64


16 лошадей попадали в больницу дважды, 268 - только 1 раз. Смертность лошадей не зависит от количества госпитализаций

In [13]:
df['rectal_temperature'].isna().sum()

60

In [14]:
print(df['rectal_temperature'].mean()) #среднее арифметическое
print(df['rectal_temperature'].median()) #медиана
print(df['rectal_temperature'].mode()[0]) #мода
print(df['rectal_temperature'].std()) #стаднартное отклонение
print(df['rectal_temperature'].var()) #дисперсия (квадрат отклонения)
print(df['rectal_temperature'].min())
print(df['rectal_temperature'].max())

38.16791666666669
38.2
38.0
0.7322886641121578
0.5362466875871686
35.4
40.8


In [15]:
print(df[df['rectal_temperature'] == df['rectal_temperature'].median()]['outcome'].value_counts())
print(df[df['rectal_temperature'] == df['rectal_temperature'].mode()[0]]['outcome'].value_counts())
print(df[df['rectal_temperature'].isna()]['outcome'].value_counts())

lived         14
died           1
euthanized     1
Name: outcome, dtype: int64
lived         17
died           6
euthanized     2
nan            1
Name: outcome, dtype: int64
lived         26
died          24
euthanized    10
Name: outcome, dtype: int64


Для пропущенных значений характерно большое количество смертей. Для сохранения общей картины, избавиться от этих значений нельзя.

Проанализируем выбросы

In [16]:
q1 = df['rectal_temperature'].quantile(0.25)
print(q1)
q3 = df['rectal_temperature'].quantile(0.75)
print(q3)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr
remove_outliers = df[df['rectal_temperature'].between(lower_bound, upper_bound, inclusive=True)].sort_values('outcome')
rt_outliers = pd.concat([df, remove_outliers]).drop_duplicates(keep=False)
rt_outliers['outcome'].value_counts()

37.8
38.5


lived         30
died          30
euthanized    14
Name: outcome, dtype: int64

А вот выбросы показывают аналогичную для пропущенных значений картину. Очевидно, "выброс" температуры является показателем тяжёлого состояния лошади. Согласно описанию, пониженная температура может свидетельствовать о шоковом состоянии, а повышенная - об инфекции

In [17]:
print(df[df['rectal_temperature'] <= lower_bound]['outcome'].value_counts())
print(df[df['rectal_temperature'] >= upper_bound]['outcome'].value_counts())

died          3
lived         2
euthanized    2
Name: outcome, dtype: int64
died          3
lived         2
euthanized    2
Name: outcome, dtype: int64


Для поиска пропущенных значений обратимся к температуре конечностей. Низкая температура указывает на шоковое состояние, а высокая - указывать на повышенную ректальную температуру. Как следствие, между ними должна сохраняться корелляция. Проверим

In [18]:
df['temperature_of_extremities'] = (df['temperature_of_extremities'].apply(str).replace('1.0', 'Normal')
                                    .replace('2.0', 'Warm')
                                    .replace('3.0', 'Cool')
                                    .replace('4.0', 'Cold'))
df['temperature_of_extremities'].value_counts()

Cool      109
Normal     78
nan        56
Warm       30
Cold       27
Name: temperature_of_extremities, dtype: int64

In [19]:
print(df[df['temperature_of_extremities']=='Normal']['rectal_temperature'].mean()) #среднее арифметическое
print(df[df['temperature_of_extremities']=='Normal']['rectal_temperature'].median()) #медиана
print(df[df['temperature_of_extremities']=='Normal']['rectal_temperature'].mode()[0]) #мода
print(df[df['temperature_of_extremities']=='Normal']['rectal_temperature'].std()) #стаднартное отклонение
print(df[df['temperature_of_extremities']=='Normal']['rectal_temperature'].var()) #дисперсия (квадрат отклонения)
print(df[df['temperature_of_extremities']=='Normal']['rectal_temperature'].min())
print(df[df['temperature_of_extremities']=='Normal']['rectal_temperature'].max())

38.12638888888889
38.150000000000006
37.8
0.5352353847214355
0.2864769170579031
36.5
39.9


In [20]:
print(df[df['temperature_of_extremities']=='Warm']['rectal_temperature'].mean()) #среднее арифметическое
print(df[df['temperature_of_extremities']=='Warm']['rectal_temperature'].median()) #медиана
print(df[df['temperature_of_extremities']=='Warm']['rectal_temperature'].mode()[0]) #мода
print(df[df['temperature_of_extremities']=='Warm']['rectal_temperature'].std()) #стаднартное отклонение
print(df[df['temperature_of_extremities']=='Warm']['rectal_temperature'].var()) #дисперсия (квадрат отклонения)
print(df[df['temperature_of_extremities']=='Warm']['rectal_temperature'].min())
print(df[df['temperature_of_extremities']=='Warm']['rectal_temperature'].max())

38.20384615384615
38.2
38.6
0.6212766013496847
0.385984615384615
36.9
39.3


In [21]:
print(df[df['temperature_of_extremities']=='Cool']['rectal_temperature'].mean()) #среднее арифметическое
print(df[df['temperature_of_extremities']=='Cool']['rectal_temperature'].median()) #медиана
print(df[df['temperature_of_extremities']=='Cool']['rectal_temperature'].mode()[0]) #мода
print(df[df['temperature_of_extremities']=='Cool']['rectal_temperature'].std()) #стаднартное отклонение
print(df[df['temperature_of_extremities']=='Cool']['rectal_temperature'].var()) #дисперсия (квадрат отклонения)
print(df[df['temperature_of_extremities']=='Cool']['rectal_temperature'].min())
print(df[df['temperature_of_extremities']=='Cool']['rectal_temperature'].max())

38.1875
38.1
38.0
0.9213879184157618
0.8489556962025306
35.4
40.8


In [22]:
print(df[df['temperature_of_extremities']=='Cold']['rectal_temperature'].mean()) #среднее арифметическое
print(df[df['temperature_of_extremities']=='Cold']['rectal_temperature'].median()) #медиана
print(df[df['temperature_of_extremities']=='Cold']['rectal_temperature'].mode()[0]) #мода
print(df[df['temperature_of_extremities']=='Cold']['rectal_temperature'].std()) #стаднартное отклонение
print(df[df['temperature_of_extremities']=='Cold']['rectal_temperature'].var()) #дисперсия (квадрат отклонения)
print(df[df['temperature_of_extremities']=='Cold']['rectal_temperature'].min())
print(df[df['temperature_of_extremities']=='Cold']['rectal_temperature'].max())

38.455
38.349999999999994
37.5
0.626162077555341
0.3920789473684209
37.5
39.4


In [23]:
print(df[df['temperature_of_extremities']=='nan']['rectal_temperature'].mean()) #среднее арифметическое
print(df[df['temperature_of_extremities']=='nan']['rectal_temperature'].median()) #медиана
print(df[df['temperature_of_extremities']=='nan']['rectal_temperature'].mode()[0]) #мода
print(df[df['temperature_of_extremities']=='nan']['rectal_temperature'].std()) #стаднартное отклонение
print(df[df['temperature_of_extremities']=='nan']['rectal_temperature'].var()) #дисперсия (квадрат отклонения)
print(df[df['temperature_of_extremities']=='nan']['rectal_temperature'].min())
print(df[df['temperature_of_extremities']=='nan']['rectal_temperature'].max())

38.04285714285713
38.05
38.3
0.7184478042187619
0.5161672473867605
36.0
39.5


В группах, где указана температура конечностей, есть наиболее характерные значения моды, которые указывают на корелляцию между температурой конечностей и ректальной температурой. Разобьём пропуски на квантили и заполним их соответствующими значениями.

In [24]:
q1 = df[df['temperature_of_extremities']=='nan']['rectal_temperature'].quantile(0.25)
q2 = df[df['temperature_of_extremities']=='nan']['rectal_temperature'].quantile(0.5)
q3 = df[df['temperature_of_extremities']=='nan']['rectal_temperature'].quantile(0.75)
df.loc[(df['temperature_of_extremities']=='nan') & (df['rectal_temperature'] <= q1), 
       'temperature_of_extremities'] = 'Cold'
df.loc[(df['temperature_of_extremities']=='nan') & (df['rectal_temperature'] <= q2), 
       'temperature_of_extremities'] = 'Cool'
df.loc[(df['temperature_of_extremities']=='nan') & (df['rectal_temperature'] <= q3), 
       'temperature_of_extremities'] = 'Normal'
df.loc[(df['temperature_of_extremities']=='nan'), 
       'temperature_of_extremities'] = 'Warm'
df['temperature_of_extremities'].value_counts()

Cool      119
Normal     88
Warm       55
Cold       38
Name: temperature_of_extremities, dtype: int64

Теперь интерполируем значения моды на пропущенные температуры

In [27]:
df['rectal_temperature'] = df.apply(
    lambda row: df[df['temperature_of_extremities']==row['temperature_of_extremities']]['rectal_temperature'].mode()[0] if pd.isna(row['rectal_temperature']) else row['rectal_temperature'], axis = 1)


In [28]:
print(df['rectal_temperature'].mean()) #среднее арифметическое
print(df['rectal_temperature'].median()) #медиана
print(df['rectal_temperature'].mode()[0]) #мода
print(df['rectal_temperature'].std()) #стаднартное отклонение
print(df['rectal_temperature'].var()) #дисперсия (квадрат отклонения)
print(df['rectal_temperature'].min())
print(df['rectal_temperature'].max())

38.16466666666669
38.1
38.0
0.6733470201816547
0.4533962095875137
35.4
40.8


Пропущенные значения заполнены, меры изменились незначительно

In [29]:
print(df['pulse'].mean()) #среднее арифметическое
print(df['pulse'].median()) #медиана
print(df['pulse'].mode()[0]) #мода
print(df['pulse'].std()) #стаднартное отклонение
print(df['pulse'].var()) #дисперсия (квадрат отклонения)
print(df['pulse'].min())
print(df['pulse'].max())
print(df['pulse'].isna().sum())

71.91304347826087
64.0
48.0
28.630556660735003
819.7087747035575
30.0
184.0
24


Для пульса характерен большой размер стандартных отклонений. При этом, основных факторов два:
* Для взрослых лошадей нормальный пульс - 30-40, но у спортивных лошадей он может быть пониженным до 20-25
* Повышенный пульс характерен для шока или для болевого синдрома

Для начала нормализуем поле pain

In [31]:
df['pain'] = (df['pain'].apply(str)
              .replace('1.0', 'no pain')
              .replace('2.0', 'depressed')
              .replace('3.0', 'mild')
              .replace('4.0', 'severe')
              .replace('5.0', 'cont severe'))
df['pain'].value_counts()

mild           67
depressed      59
nan            55
cont severe    42
severe         39
no pain        38
Name: pain, dtype: int64

Также обращаем внимание на зависимость пульса от возраста

In [37]:
print(df[df['age'] == 'young']['pulse'].mean())
print(df[df['age'] == 'young']['pulse'].median())
print(df[df['age'] == 'young']['pulse'].std())
print(df[df['age'] == 'adult']['pulse'].mean())
print(df[df['age'] == 'adult']['pulse'].median())
print(df[df['age'] == 'adult']['pulse'].std())

123.36363636363636
124.0
28.62823357414421
67.45669291338582
60.0
23.933952566649694


Очевидно, при такой разнице, их надо рассматривать как две отдельные группы

Рассмотрим зависимость пульса от температуры и боли

In [68]:
df_a = df.loc[df['age'] == 'adult', ['pain', 'temperature_of_extremities', 'pulse']]
pd.concat([
    df_a.groupby(['temperature_of_extremities', 'pain']).mean(),
    df_a.groupby(['temperature_of_extremities', 'pain']).median(),
    df_a.groupby(['temperature_of_extremities', 'pain']).std()
], axis=1)

pulse  pulse      pulse
temperature_of_extremities pain                                    
Cold                       cont severe  95.600000   90.0  25.234896
                           depressed    94.000000   96.0  28.936713
                           mild         59.000000   60.0  17.606817
                           nan          66.800000   48.0  31.399929
                           severe       77.750000   73.5  18.661458
Cool                       cont severe  87.200000   91.0  18.642340
                           depressed    79.041667   79.0  24.702542
                           mild         62.100000   60.0  17.332355
                           nan          64.692308   60.0  21.437446
                           no pain      45.142857   48.0   4.298394
                           severe       80.333333   81.0  27.655443
Normal                     cont severe  64.400000   70.0  16.456002
                           depressed    62.933333   56.0  26.628306
                           mild         56.916667   52.0  14.397816
                           nan          61.545455   52.0  22.357834
                           no pain      48.875000   48.0   5.512108
                           severe       64.571429   64.0  13.986388
Warm                       cont severe  64.500000   64.5   0.707107
                           depressed    62.000000   72.0  17.320508
                           mild         58.600000   54.0  17.664780
                           nan          64.750000   64.0  29.236719
                           no pain      51.000000   48.0   7.438638
                           severe       90.666667   96.0  20.526406

внутри каждой группы есть корелляция, но разброс слишком высок

In [78]:
df_a = df.loc[df['age'] == 'adult', ['pain', 'temperature_of_extremities', 'pulse']]
q1 = df_a['pulse'].quantile(0.25)
q3 = df_a['pulse'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr
remove_outliers = df_a[df_a['pulse'].between(lower_bound, upper_bound, inclusive=True)].sort_values('pulse')
pd.concat([
    remove_outliers.groupby(['temperature_of_extremities', 'pain']).mean(),
    remove_outliers.groupby(['temperature_of_extremities', 'pain']).median(),
    remove_outliers.groupby(['temperature_of_extremities', 'pain']).std()
], axis=1)

pulse  pulse      pulse
temperature_of_extremities pain                                    
Cold                       cont severe  95.600000   90.0  25.234896
                           depressed    94.000000   96.0  28.936713
                           mild         59.000000   60.0  17.606817
                           nan          66.800000   48.0  31.399929
                           severe       77.750000   73.5  18.661458
Cool                       cont severe  87.200000   91.0  18.642340
                           depressed    79.041667   79.0  24.702542
                           mild         62.100000   60.0  17.332355
                           nan          64.692308   60.0  21.437446
                           no pain      45.142857   48.0   4.298394
                           severe       76.823529   80.0  24.021437
Normal                     cont severe  64.400000   70.0  16.456002
                           depressed    62.933333   56.0  26.628306
                           mild         56.916667   52.0  14.397816
                           nan          61.545455   52.0  22.357834
                           no pain      48.875000   48.0   5.512108
                           severe       64.571429   64.0  13.986388
Warm                       cont severe  64.500000   64.5   0.707107
                           depressed    62.000000   72.0  17.320508
                           mild         58.600000   54.0  17.664780
                           nan          64.750000   64.0  29.236719
                           no pain      51.000000   48.0   7.438638
                           severe       90.666667   96.0  20.526406

В полученных результатах можно заметить, что при отсутствии боли пульс стабилен при любой температуре конечностей

In [ ]:
q1 = df['rectal_temperature'].quantile(0.25)
print(q1)
q3 = df['rectal_temperature'].quantile(0.75)
print(q3)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr
remove_outliers = df[df['rectal_temperature'].between(lower_bound, upper_bound, inclusive=True)].sort_values('outcome')
rt_outliers = pd.concat([df, remove_outliers]).drop_duplicates(keep=False)
rt_outliers['outcome'].value_counts()

In [20]:
q1 = df['pulse'].quantile(0.25)
q3 = df['pulse'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr
remove_outliers = df[df['pulse'].between(lower_bound, upper_bound, inclusive=True)].sort_values('outcome')
rt_outliers = pd.concat([df, remove_outliers]).drop_duplicates(keep=False)
rt_outliers['outcome'].value_counts()

died          15
lived         13
euthanized     1
Name: outcome, dtype: int64

В случае с лошадьми с аномально низким или высоким пульсом ситуация аналогичная - более половины лошадей умерли. Эти данные следует оставить

In [21]:
df[df['pulse'].isna()]['outcome'].value_counts()

lived         12
died          11
euthanized     1
Name: outcome, dtype: int64

In [22]:
df['pulse'].value_counts().head(5)

48.0    28
60.0    25
40.0    18
88.0    12
52.0    12
Name: pulse, dtype: int64

In [23]:
print(df[df['pulse'] == df['pulse'].median()]['outcome'].value_counts())
print(df[df['pulse'] == df['pulse'].mode()[0]]['outcome'].value_counts())

lived    6
died     2
Name: outcome, dtype: int64
lived         25
euthanized     2
nan            1
Name: outcome, dtype: int64


Для лошадей с неуказанным пульсом сохраняется репрезентативность выборки. Для сохранения статистики проставим для них медианное значение, поскольку разница между средним и медианным значением гораздо ниже, чем между средним и модой. Кроме того, значение моды слишком сильно сказывается на выживаемости лошадей.

In [24]:
print(df['respiratory_rate'].isna().sum())
print(df['respiratory_rate'].mean()) #среднее арифметическое
print(df['respiratory_rate'].median()) #медиана
print(df['respiratory_rate'].mode()[0]) #мода
print(df['respiratory_rate'].std()) #стаднартное отклонение
print(df['respiratory_rate'].var()) #дисперсия (квадрат отклонения)
print(df['respiratory_rate'].min())
print(df['respiratory_rate'].max())

58
30.417355371900825
24.5
20.0
17.642231385134664
311.24832824663054
8.0
96.0


In [25]:
print(df[df['respiratory_rate'].isna()]['outcome'].value_counts())
print(df[df['respiratory_rate'] == 24]['outcome'].value_counts())
print(df[df['respiratory_rate'] == df['respiratory_rate'].mode()[0]]['outcome'].value_counts())

lived         31
died          19
euthanized     8
Name: outcome, dtype: int64
lived         12
died           9
euthanized     6
Name: outcome, dtype: int64
lived         14
euthanized     8
died           5
nan            1
Name: outcome, dtype: int64


In [26]:
df['respiratory_rate'].value_counts()

20.0    28
24.0    27
16.0    22
12.0    19
30.0    19
40.0    17
36.0    16
28.0    13
32.0    11
18.0     8
48.0     6
14.0     4
60.0     4
35.0     3
68.0     3
10.0     3
80.0     3
44.0     3
42.0     3
90.0     2
84.0     2
70.0     2
21.0     2
96.0     2
22.0     2
51.0     2
9.0      2
50.0     2
72.0     1
23.0     1
15.0     1
34.0     1
25.0     1
8.0      1
52.0     1
58.0     1
13.0     1
66.0     1
26.0     1
88.0     1
Name: respiratory_rate, dtype: int64

In [27]:
df['respiratory_rate'].fillna(24, inplace=True)

NA значения попадают в общую выборку по результатам лечения. Медиана по неясной для меня причине не является значением из выборки. Ближайшее к ней значение - 24, даёт верную статистику по исходам, поэтому можно использовать это значение для заполнения пропущенных значений.

In [33]:
df['temperature_of_extremities'] = (df['temperature_of_extremities'].apply(str)
                                    .replace('1.0', 'Normal')
                                    .replace('2.0', 'Warm')
                                    .replace('3.0', 'Cool')
                                    .replace('4.0', 'Cold'))
df['temperature_of_extremities'].value_counts()

Cool      109
Normal     78
nan        56
Warm       30
Cold       27
Name: temperature_of_extremities, dtype: int64

## Дополнительная часть (необязательная)

Выполнить задания 1-3 для всего набора данных.

#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий [GitHub](https://github.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам
Прежде чем задать вопрос, попробуйте найти ответ в интернете. Навык самостоятельного поиска информации — один из важнейших. Каждый практикующий специалист любого уровня делает это ежедневно.

Сформулируйте вопрос по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности прикрепите к вопросу скриншоты либо ссылки на код. Не выкладывайте все решение, оставляйте только проблемный и воспроизводимый участок кода.